In [71]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 3s (105 kB/s)
Reading p

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import math

In [73]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [74]:
#arizona = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Arizona.csv'
idaho = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Idaho.csv'
illinois = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Illinois.csv'
indiana = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Indiana.csv'
iowa = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Iowa.csv'
kansas = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Kansas.csv'
kentucky = 'https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/States_raw_data/Kentucky.csv'
states = [idaho, illinois, indiana, iowa, kansas, kentucky]

In [75]:
hospitals = pd.read_csv(illinois)
hospitals = hospitals[hospitals['AMC_No'].notna()]
hospitals['AMC_No'] = hospitals["AMC_No"].astype(int)

baseurl = 'https://freida.ama-assn.org/institution/'
URLS = []
for index, i in hospitals.iterrows():
  URLS.append(str(baseurl + str(i['AMC_No'])))
hospitals['URLS'] = URLS

In [76]:
len(hospitals)

302

In [77]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("enable-automation")
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-browser-side-navigation")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup



In [78]:
# d = driversetup()
# relationship = []
# for index, i in hospitals.iterrows():
#   url = i['URLS']
#   s = pagesource(url, d)
#   med_school = list(s.findAll('small', class_="bold"))
#   print(i.Med_center, index, relationship, len(med_school))
#   for i in range(len(med_school)):
#     a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    
#     for i in range(len(a)):
#       rel = a[i].findAll('div', class_='')[0].text
#       if rel not in relationship:
#         print(med_school[i].text, rel)
#         relationship.append(rel)
# relationship

In [ ]:
mc = []
grad = []
mcl = []
add = []
d = driversetup()
for index, i in hospitals.iterrows():
  url = i['URLS']
  s = pagesource(url, d)
  med_school = list(s.findAll('small', class_="bold"))
  address = list(s.findAll('div', class_="details__address ng-star-inserted")) 
  if len(med_school) == 0:
    mc.append("")
    mcl.append("")
    grad.append("")
  elif len(med_school) == 1:
    a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    name = a[0].findAll('small', class_='bold')[0].text
    relationship = a[0].findAll('div', class_='')[0].text
    if 'Limited' in relationship:
      mcl.append(name)
      mc.append("")
      grad.append("")
    elif 'Major' in relationship:
      mc.append(name)
      mcl.append("")
      grad.append("")
    elif 'Graduate' in relationship:
      mc.append("")
      mcl.append("")
      grad.append(name)
    else:
      print("NEW RELATIONSHIP FOUND : ", name)
      break
  elif len(med_school) > 1:   
      major = []
      limited = []
      graduate = []
      a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
      for i in range(len(a)):
        name = a[i].findAll('small', class_='bold')[0].text
        relationship = a[i].findAll('div', class_='')[0].text
        if 'Limited' in relationship:
          limited.append(name)
        elif 'Graduate' in relationship:
          graduate.append(name)
        elif 'Major' in relationship:
          major.append(name)
        else:
          print("New relationship found")
          break 
      if major == []:
        major = ''
      if limited == []:
        limited = ''
      if grad == []:
        grad = ''
      mc.append(major)
      mcl.append(limited)
      grad.append(graduate)
  if len(address) != 0:
    address[0].br.string = ', '
    add.append(address[0].text)
  else:
    add.append("")
  print(index, 'mc', len(mc), mc[index], 'mcl', len(mcl), mcl[index], 'grad', len(grad), grad[index], 'add', len(add), add[index])
  if len(mc) > (index+1):
    break



0 mc 1  mcl 1  grad 1  add 1 60 E Delaware Pl, Chicago, IL 60611
1 mc 2  mcl 2  grad 2  add 2 901 Southwind Rd, Springfield, IL 62703
2 mc 3  mcl 3  grad 3  add 3 4944 95th St, Oak Lawn, IL 60435
3 mc 4  mcl 4  grad 4  add 4 222 N Canal St, Chicago, IL 60606
4 mc 5  mcl 5  grad 5  add 5 1 E Erie St, Chicago, IL 60611
5 mc 6 ['Midwestern Univ Chicago Coll of Osteopathic Med'] mcl 6 ['Rush Med Coll of Rush Univ Med Ctr', 'Loyola Univ of Chicago Stritch Sch of Med', 'Kansas City Univ of Med and Biosciences Coll of Osteopathic Med'] grad 6 ['Univ of Chicago Div of the Biological Sciences Pritzker Sch of Med'] add 6 5101 S Willow Springs Rd, LaGrange, IL 60525
6 mc 7 ['Univ of Illinois Coll of Med', 'Loyola Univ of Chicago Stritch Sch of Med', 'Midwestern Univ Chicago Coll of Osteopathic Med'] mcl 7 ['Midwestern Univ Arizona Coll of Osteopathic Med', 'Rush Med Coll of Rush Univ Med Ctr', 'Lincoln Memorial Univ DeBusk Coll of Osteopathic Med'] grad 7 ['Univ of Chicago Div of the Biological S

In [ ]:
print(len(mc),len(mcl),len(grad),len(add))
hospitals['Affiliation_Major'] = mc
hospitals['Affiliation_Limited'] = mcl
hospitals['Affiliation_Graduate'] = grad
hospitals['Medical_Center_Address'] = add

In [ ]:
hospitals.to_csv('Kentucky_crosswalk.csv')